## Problem: You have 3D images but not enough GPU memory. ##

**Method 1: Train Different Networks on Different Views and Combine**

**tensorflow**

In [17]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Define a simple 2D CNN model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3)  # Adjust kernel size here
        self.conv2 = nn.Conv2d(32, 64, 3)  # Adjust kernel size here
        self.conv3 = nn.Conv2d(64, 64, 3)  # Adjust kernel size here
        self.pool = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(64 * 14 * 14, 64)  # Adjust input size here
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = self.pool(nn.functional.relu(self.conv3(x)))
        x = torch.flatten(x, 1)
        x = nn.functional.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))  # Use torch.sigmoid instead of nn.functional.sigmoid
        return x

# Generate synthetic data for demonstration
def generate_data(num_samples, height, width, num_channels):
    return torch.rand((num_samples, num_channels, height, width)), torch.randint(0, 2, (num_samples,))

num_samples = 100
height, width, num_channels = 64, 64, 1
num_views = 3

# Generate synthetic data (replace with your actual data loading)
X, y = generate_data(num_samples, height, width, num_channels)

# Split data into axial, sagittal, and coronal views
X_axial = X[:, :, :, 0:1]  # Assuming first channel represents axial view
X_sagittal = X[:, :, :, 1:2]  # Assuming second channel represents sagittal view
X_coronal = X[:, :, :, 2:3]  # Assuming third channel represents coronal view

# Train individual models on different views
models_list = []
criterion = nn.BCELoss()
optimizer = optim.Adam

for X_view in [X_axial, X_sagittal, X_coronal]:
    model = CNN()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    for epoch in range(5):
        optimizer.zero_grad()
        outputs = model(X_view)
        loss = criterion(outputs, y.unsqueeze(1).float())
        loss.backward()
        optimizer.step()
    models_list.append(model)

# Combine outputs of individual models (e.g., by averaging)
def combine_outputs(models, X):
    predictions = torch.zeros((len(X), 1))
    for model in models:
        predictions += model(X)
    return predictions / len(models)

# Combine outputs for each view
X_combined = torch.cat((X_axial, X_sagittal, X_coronal), dim=1)
combined_predictions = combine_outputs(models_list, X_combined)

# Evaluate combined predictions (for demonstration)
with torch.no_grad():
    combined_loss = criterion(combined_predictions, y.unsqueeze(1).float())
    combined_accuracy = ((combined_predictions > 0.5).float() == y.unsqueeze(1).float()).float().mean()

print("Combined Model - Loss: {}, Accuracy: {}".format(combined_loss.item(), combined_accuracy.item()))


RuntimeError: Calculated padded input size per channel: (64 x 1). Kernel size: (3 x 3). Kernel size can't be greater than actual input size